In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
import os
import tarfile

df = pd.read_csv('train.csv', sep='|')
df = df.melt().dropna().rename({'value':'text', 'variable': 'label'},axis=1)
df["label"] = pd.to_numeric(df["label"]) - 1
df = pd.concat([df.iloc[:, -1], df.iloc[:, 0]], axis=1)

train_df, test_df = train_test_split(df, test_size=0.1)

In [ ]:
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 15}

model = ClassificationModel(
    "bert", "Geotrend/distilbert-base-ru-cased",
    num_labels=39,
    args=train_args,
)

In [ ]:
model.train_model(train_df)

In [ ]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

In [ ]:
def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

pack_model('outputs','bert')